# Joda HT

### Useful links

Spotify offers very clear documentation on how to work with their API. 
[Spotify API Documentation](https://developer.spotify.com/)

Console page lets you test making requests 
[API Console](https://developer.spotify.com/console/)

Quick guide on how to use the API: [Guide](https://stmorse.github.io/journal/spotify-api.html)

Config file includes the Client ID and Secret variables that should be kept private. These are needed in order to access the API.

If you want to run this notebook yourself: 
- you need a spotify account (free or premium, doesn't matter)
- sign in on https://developer.spotify.com/
- Go to Dashboard -> Create an app -> The app page will show the ID and Secret that you need to save to the config file

### Fetch and save data from the Spotify API

In [3]:
import config
import requests
import pandas as pd

#Get client data from the config file
clientID = config.clientID
clientSEC = config.clientSEC

# Get the token with post request
tokenURL = 'https://accounts.spotify.com/api/token'

response = requests.post(tokenURL, {
    'grant_type': 'client_credentials',
    'client_id': clientID,
    'client_secret': clientSEC,
})

responseData = response.json()
token = responseData['access_token']

#Information for the get request
baseURL = 'https://api.spotify.com/v1/'
playlistID = '37i9dQZF1DX18jTM2l2fJY'

headers = {
    'Authorization': 'Bearer {token}'.format(token=token)
}

#Get the top 2021 playlist
top2021 = requests.get(baseURL + 'playlists/' + playlistID, headers=headers)
top2021 = top2021.json()
tracks = top2021["tracks"]["items"]

stuff = []

#Save song data from the get request
for item in tracks:
    #Get basic track info
    artist = item["track"]["artists"][0]["name"]
    track = item["track"]["name"]
    album = item["track"]["album"]["name"]

    #Get audio features
    songID = item["track"]["id"]
    songData = requests.get(baseURL + 'audio-features?ids=' + songID, headers=headers)
    songData =  songData.json()
    features = songData["audio_features"][0]

    features.update({
    'artist': artist,
    'track': track, 
    'album': album,
    })

    stuff.append(features)

df = pd.DataFrame(stuff)
df.to_csv('data.csv')



### Clean the data

In [4]:
import pandas as pd

df = pd.read_csv('data.csv')

df.head(5)

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist,track,album
0,0,0.561,0.431,10,-8.810,1,0.0578,0.76800,0.000014,0.1060,...,audio_features,5wANPM4fQCJwkGd4rN57mH,spotify:track:5wANPM4fQCJwkGd4rN57mH,https://api.spotify.com/v1/tracks/5wANPM4fQCJw...,https://api.spotify.com/v1/audio-analysis/5wAN...,242013,4,Olivia Rodrigo,drivers license,SOUR
1,1,0.593,0.503,8,-6.725,0,0.2200,0.29300,0.000000,0.4050,...,audio_features,1SC5rEoYDGUK4NfG82494W,spotify:track:1SC5rEoYDGUK4NfG82494W,https://api.spotify.com/v1/tracks/1SC5rEoYDGUK...,https://api.spotify.com/v1/audio-analysis/1SC5...,137704,4,Lil Nas X,MONTERO (Call Me By Your Name),MONTERO
2,2,0.591,0.764,1,-5.484,1,0.0483,0.03830,0.000000,0.1030,...,audio_features,5PjdY0CKGZdEuoNab3yDmX,spotify:track:5PjdY0CKGZdEuoNab3yDmX,https://api.spotify.com/v1/tracks/5PjdY0CKGZdE...,https://api.spotify.com/v1/audio-analysis/5Pjd...,141806,4,The Kid LAROI,STAY (with Justin Bieber),F*CK LOVE 3: OVER YOU
3,3,0.563,0.664,9,-5.044,1,0.1540,0.33500,0.000000,0.0849,...,audio_features,4ZtFanR9U6ndgddUvNcjcG,spotify:track:4ZtFanR9U6ndgddUvNcjcG,https://api.spotify.com/v1/tracks/4ZtFanR9U6nd...,https://api.spotify.com/v1/audio-analysis/4ZtF...,178147,4,Olivia Rodrigo,good 4 u,SOUR
4,4,0.702,0.825,6,-3.787,0,0.0601,0.00883,0.000000,0.0674,...,audio_features,5nujrmhLynf4yMoMtj8AQF,spotify:track:5nujrmhLynf4yMoMtj8AQF,https://api.spotify.com/v1/tracks/5nujrmhLynf4...,https://api.spotify.com/v1/audio-analysis/5nuj...,203064,4,Dua Lipa,Levitating (feat. DaBaby),Future Nostalgia
